# ARCO Algorithm Demo: XRD Pattern Analysis

This notebook demonstrates the **ARCO (Arcogram with Rational Coherence Index)** algorithm for analyzing periodicity in XRD diffraction patterns.

## What is ARCO?

ARCO converts 1-D signals (e.g., XRD intensity vs. 2θ) into interpretable fingerprints by:
1. Computing local power spectra using sliding windows
2. Integrating spectral power around **rational frequency anchors** (e.g., 1/2, 1/3, 1/7)
3. Producing:
   - **ARCO-print**: Fixed-length feature vector
   - **RCI**: Rational Coherence Index (scalar measure of periodicity)
   - **ARCO-3D**: Position-resolved arcogram

Rational anchors correspond to small-integer periodicities common in crystalline materials.

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import matplotlib.pyplot as plt
from nomad_auto_xrd.lib.arco_rci import (
    generate_anchors, 
    ARCO, 
    compute_derivative_track,
    resample_uniform,
    interpret_rational
)

# Set random seed for reproducibility
np.random.seed(42)

print("ARCO library imported successfully!")

## 1. Generate Rational Anchors

First, we generate the rational frequency anchors up to denominator `Qmax`.

In [ ]:
# Generate anchors for XRD (use higher Qmax for longer periodicities)
Qmax = 40
anchors = generate_anchors(Qmax=Qmax)

print(f"Generated {len(anchors)} rational anchors for Qmax={Qmax}")
print(f"\nFirst 10 anchors: {anchors[:10]}")
print(f"Last 10 anchors: {anchors[-10:]}")

# Visualize anchor distribution
plt.figure(figsize=(12, 3))
plt.scatter(anchors, [1]*len(anchors), alpha=0.6, s=20)
plt.xlabel('Frequency (cycles/sample)')
plt.ylabel('Anchors')
plt.title(f'Distribution of {len(anchors)} Rational Frequency Anchors (Qmax={Qmax})')
plt.yticks([])
plt.grid(True, alpha=0.3)
plt.xlim(0, 0.5)
plt.show()

## 2. Synthetic XRD Pattern: Uniform Peak Spacing

Create a synthetic XRD pattern with uniformly spaced peaks (like a simple lattice).

In [ ]:
# Create synthetic XRD pattern with uniform peak spacing
n_points = 2048
two_theta = np.linspace(10, 80, n_points)

# Uniform peaks every 5 degrees (simple cubic-like)
intensity_uniform = np.zeros(n_points)
peak_spacing = 5.0  # degrees
peak_positions = np.arange(15, 75, peak_spacing)

for pos in peak_positions:
    # Add Gaussian peak
    intensity_uniform += 100 * np.exp(-((two_theta - pos)**2) / 0.8)

# Add background noise
intensity_uniform += 20 + 5 * np.random.randn(n_points)
intensity_uniform = np.maximum(intensity_uniform, 0)  # No negative intensities

# Plot
plt.figure(figsize=(14, 4))
plt.plot(two_theta, intensity_uniform, linewidth=0.8)
plt.xlabel('2θ (degrees)')
plt.ylabel('Intensity (a.u.)')
plt.title('Synthetic XRD Pattern: Uniform Peak Spacing (5° intervals)')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Peak positions: {peak_positions}")

## 3. Random Peak Spacing (Low Periodicity)

Create a pattern with randomly spaced peaks for comparison.

In [ ]:
# Random peak positions
intensity_random = np.zeros(n_points)
random_peaks = np.random.uniform(15, 75, 12)

for pos in random_peaks:
    intensity_random += 100 * np.exp(-((two_theta - pos)**2) / 0.8)

intensity_random += 20 + 5 * np.random.randn(n_points)
intensity_random = np.maximum(intensity_random, 0)

# Plot comparison
fig, axes = plt.subplots(2, 1, figsize=(14, 7), sharex=True)

axes[0].plot(two_theta, intensity_uniform, linewidth=0.8, label='Uniform spacing')
axes[0].set_ylabel('Intensity (a.u.)')
axes[0].set_title('Uniform Peak Spacing (Periodic)')
axes[0].grid(True, alpha=0.3)

axes[1].plot(two_theta, intensity_random, linewidth=0.8, color='orange', label='Random spacing')
axes[1].set_xlabel('2θ (degrees)')
axes[1].set_ylabel('Intensity (a.u.)')
axes[1].set_title('Random Peak Spacing (Non-periodic)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. ARCO Analysis

Apply ARCO to both patterns and compute RCI.

In [ ]:
# Initialize ARCO analyzer
# For XRD: use window_sizes appropriate for pattern resolution
# alpha tuned for XRD patterns
arco = ARCO(
    anchors=anchors,
    window_sizes=[128, 256],
    hop_fraction=0.25,
    alpha=0.5,  # Tune this parameter
    apply_hann=True
)

print("ARCO analyzer initialized")
print(f"  Window sizes: {arco.window_sizes}")
print(f"  Alpha: {arco.alpha}")
print(f"  Number of anchors: {len(arco.anchors)}")

In [ ]:
# Compute RCI for both patterns
major_q = 20  # Consider rationals with q <= 20 as "major"

rci_uniform = arco.compute_rci({'intensity': intensity_uniform}, major_q=major_q)
rci_random = arco.compute_rci({'intensity': intensity_random}, major_q=major_q)

print(f"\n=== Rational Coherence Index (RCI) ===")
print(f"Uniform spacing: RCI = {rci_uniform:.4f}")
print(f"Random spacing:  RCI = {rci_random:.4f}")
print(f"\nDifference: {rci_uniform - rci_random:.4f}")
print(f"\nHigher RCI → stronger periodic structure")

## 5. Arc Power Analysis

Examine which rational frequencies dominate each pattern.

In [ ]:
# Compute arc powers for uniform pattern
arc_matrix_uniform = arco.compute_track_arcs(intensity_uniform, window_size=256)
avg_arc_uniform = np.mean(arc_matrix_uniform, axis=0)

# Compute arc powers for random pattern
arc_matrix_random = arco.compute_track_arcs(intensity_random, window_size=256)
avg_arc_random = np.mean(arc_matrix_random, axis=0)

# Get top rationals
top_uniform = arco.get_top_rationals(avg_arc_uniform, top_k=10)
top_random = arco.get_top_rationals(avg_arc_random, top_k=10)

print("Top 10 rational frequencies (Uniform pattern):")
for freq, power, denom in top_uniform:
    print(f"  {freq:.4f} (q={denom:2d}): power = {power:.4f} - {interpret_rational(freq)}")

print("\nTop 10 rational frequencies (Random pattern):")
for freq, power, denom in top_random:
    print(f"  {freq:.4f} (q={denom:2d}): power = {power:.4f} - {interpret_rational(freq)}")

In [ ]:
# Visualize arc power spectra
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Uniform pattern
axes[0].bar(anchors, avg_arc_uniform, width=0.005, alpha=0.7)
axes[0].set_xlabel('Rational Frequency')
axes[0].set_ylabel('Arc Power')
axes[0].set_title(f'ARCO Spectrum: Uniform Peak Spacing (RCI={rci_uniform:.4f})')
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_xlim(0, 0.5)

# Highlight top 5
for freq, power, _ in top_uniform[:5]:
    axes[0].axvline(freq, color='red', alpha=0.3, linestyle='--', linewidth=0.8)

# Random pattern
axes[1].bar(anchors, avg_arc_random, width=0.005, alpha=0.7, color='orange')
axes[1].set_xlabel('Rational Frequency')
axes[1].set_ylabel('Arc Power')
axes[1].set_title(f'ARCO Spectrum: Random Peak Spacing (RCI={rci_random:.4f})')
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].set_xlim(0, 0.5)

plt.tight_layout()
plt.show()

## 6. ARCO-Print: Feature Vector

Compute the ARCO fingerprint for each pattern.

In [ ]:
# Compute ARCO-prints
arco_print_uniform = arco.compute_arco_print({'intensity': intensity_uniform})
arco_print_random = arco.compute_arco_print({'intensity': intensity_random})

print(f"ARCO-print dimensions: {len(arco_print_uniform)}")
print(f"  = {len(arco.window_sizes)} window sizes × {len(anchors)} anchors × 1 track")

# Compute L1 distance between fingerprints
l1_distance = np.sum(np.abs(arco_print_uniform - arco_print_random))
print(f"\nL1 distance between fingerprints: {l1_distance:.4f}")

# Visualize fingerprints
fig, axes = plt.subplots(2, 1, figsize=(14, 6))

axes[0].plot(arco_print_uniform, linewidth=0.5, alpha=0.8)
axes[0].set_ylabel('Arc Power')
axes[0].set_title('ARCO-Print: Uniform Pattern')
axes[0].grid(True, alpha=0.3)

axes[1].plot(arco_print_random, linewidth=0.5, alpha=0.8, color='orange')
axes[1].set_xlabel('Feature Index')
axes[1].set_ylabel('Arc Power')
axes[1].set_title('ARCO-Print: Random Pattern')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Multi-Track Analysis (with Derivative)

Use both intensity and derivative tracks for richer features.

In [ ]:
# Compute derivative tracks
deriv_uniform = compute_derivative_track(intensity_uniform)
deriv_random = compute_derivative_track(intensity_random)

# Multi-track analysis
tracks_uniform = {
    'intensity': intensity_uniform,
    'derivative': deriv_uniform
}

tracks_random = {
    'intensity': intensity_random,
    'derivative': deriv_random
}

# Compute multi-track RCI
rci_uniform_multi = arco.compute_rci(tracks_uniform, major_q=major_q)
rci_random_multi = arco.compute_rci(tracks_random, major_q=major_q)

print(f"Multi-track RCI (intensity + derivative):")
print(f"  Uniform: {rci_uniform_multi:.4f} (single-track: {rci_uniform:.4f})")
print(f"  Random:  {rci_random_multi:.4f} (single-track: {rci_random:.4f})")

# Multi-track ARCO-print
arco_print_uniform_multi = arco.compute_arco_print(tracks_uniform)
print(f"\nMulti-track ARCO-print dimension: {len(arco_print_uniform_multi)}")
print(f"  = {len(arco.window_sizes)} window sizes × {len(anchors)} anchors × 2 tracks")

## 8. ARCO-3D: Position-Resolved Analysis

Visualize how periodicity varies along the pattern.

In [ ]:
# Compute ARCO-3D (position-resolved)
arco_3d_uniform = arco.compute_arco_3d(tracks_uniform, finest_window=128)
arco_3d_random = arco.compute_arco_3d(tracks_random, finest_window=128)

print(f"ARCO-3D shape: {arco_3d_uniform.shape}")
print(f"  = (n_windows={arco_3d_uniform.shape[0]}, n_features={arco_3d_uniform.shape[1]})")

# Visualize as heatmap
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

im0 = axes[0].imshow(
    arco_3d_uniform.T,
    aspect='auto',
    cmap='viridis',
    interpolation='nearest'
)
axes[0].set_ylabel('Feature Index')
axes[0].set_title('ARCO-3D: Uniform Pattern (position × features)')
plt.colorbar(im0, ax=axes[0], label='Arc Power')

im1 = axes[1].imshow(
    arco_3d_random.T,
    aspect='auto',
    cmap='viridis',
    interpolation='nearest'
)
axes[1].set_xlabel('Window Index (Position)')
axes[1].set_ylabel('Feature Index')
axes[1].set_title('ARCO-3D: Random Pattern (position × features)')
plt.colorbar(im1, ax=axes[1], label='Arc Power')

plt.tight_layout()
plt.show()

## 9. Statistical Validation (Z-score)

Compute z-score against shuffled null model to validate significance.

In [ ]:
# Compute z-scores (this may take a moment)
print("Computing z-scores against shuffled null model...")
print("(This may take 30-60 seconds)\n")

zscore_uniform = arco.null_model_zscore(
    intensity_uniform,
    n_shuffles=30,
    preserve_composition=True
)

zscore_random = arco.null_model_zscore(
    intensity_random,
    n_shuffles=30,
    preserve_composition=True
)

print(f"Z-scores (vs. shuffled null):")
print(f"  Uniform pattern: z = {zscore_uniform:.2f}")
print(f"  Random pattern:  z = {zscore_random:.2f}")
print(f"\nInterpretation:")
print(f"  |z| > 3: Highly significant periodicity")
print(f"  |z| > 2: Significant periodicity")
print(f"  |z| < 2: Not significantly different from noise")

## 10. Summary and Applications

### Key Takeaways:

1. **RCI**: Single scalar metric quantifying periodic structure (0-1)
2. **ARCO-print**: Fixed-length feature vector for ML/similarity search
3. **ARCO-3D**: Position-resolved map for localization
4. **Top Rationals**: Interpretable frequency components

### Applications for XRD:

- **Phase identification**: Use ARCO-print for similarity-based retrieval
- **Quality control**: RCI as metric for crystallinity
- **Structure optimization**: Guide inverse design with RCI loss
- **Peak spacing analysis**: Identify characteristic periodicities

### Parameter Tuning:

- **Qmax**: 30-60 for XRD (longer periodicities)
- **window_sizes**: 128-256 for typical XRD resolution
- **alpha**: 0.3-1.0 (tune based on peak sharpness)
- **major_q**: 10-20 (determines which rationals are "major")

In [ ]:
# Final comparison table
import pandas as pd

comparison = pd.DataFrame({
    'Metric': ['RCI (single-track)', 'RCI (multi-track)', 'Z-score', 'ARCO-print L1 distance'],
    'Uniform Pattern': [f"{rci_uniform:.4f}", f"{rci_uniform_multi:.4f}", f"{zscore_uniform:.2f}", '-'],
    'Random Pattern': [f"{rci_random:.4f}", f"{rci_random_multi:.4f}", f"{zscore_random:.2f}", '-'],
    'Comparison': ['', '', '', f"{l1_distance:.4f}"]
})

print("\n=== ARCO Analysis Summary ===")
print(comparison.to_string(index=False))
print("\n✓ Demo completed successfully!")